In [1]:
import pandas as pd
from yahoofinancials import YahooFinancials
import numpy as np
import json

## FAANG Companies

In [2]:
# Pull the yearly data from the JSON file and append to yearly list

# Pass the json file, end of financial year date (JSON Key) and fields to keep 
def extract_financials(year_data, year, field_list):
    temp_list = []
    year_data['financialYear'] = year
    
    # Extract the requested fields
    for field in field_list:
        
        # Empty Field catch
        if field in year_data.keys():
            temp_list.append(year_data[field])
        else:
            temp_list.append(0)
    
    # Return the field values for the year
    return temp_list

In [3]:
companies = ['TSLA','MSFT','AAPL','AMZN','META']

income_fields = ['financialYear','totalRevenue','costOfRevenue','researchAndDevelopment',
                 'sellingGeneralAndAdministration','netInterestIncome','netIncomeCommonStockholders']

balance_fields = ['financialYear','cashCashEquivalentsAndShortTermInvestments','accountsReceivable',
                  'inventory','currentAssets','netPPE','totalNonCurrentAssets','totalAssets','accountsPayable',
                 'totalLiabilitiesNetMinorityInterest','retainedEarnings','stockholdersEquity']

In [4]:

for company in companies:
    # Create Yahoo Financial Object for information
    yf_company = YahooFinancials(company)
    # Create two empty lists to append yearly data
    income_statement_list, balance_statement_list = [], []
    
    # Extract Income and then Balance Sheet financials
    for statement_type in ['income','balance']:
        # Extract all years financial statement information in JSON format
        json_company_financials = yf_company.get_financial_stmts('annual',statement_type,reformat=False)
        
        # Get the specific End of Financial year dates
        fin_year_list = list(json_company_financials[company].keys())
        
        # Iterate over the end of financial year dates to access the yearly information
        for fin_year in fin_year_list:
            
            # Extract Income Statement information
            if statement_type == 'income':
                try:
                    # Pass the relevant year section of JSON file, end of financial year date and fields to extract
                    income_statement_list.append(extract_financials(json_company_financials[company][fin_year],
                                            fin_year, income_fields))
                except:
                    print(f'{company} has no {statement_type} information for year {fin_year}')
            
            # Extract Balance Sheet information
            elif statement_type == 'balance':
                try:
                    # Pass the relevant year section of JSON file, end of financial year date and fields to extract
                    balance_statement_list.append(extract_financials(json_company_financials[company][fin_year],
                                              fin_year, balance_fields))
                except:
                    # Error Handling, Incomplete information 
                    print(f'{company} has no {statement_type} information for {fin_year}')
    

        if statement_type == 'income':
            fin_df = pd.DataFrame(income_statement_list, columns=income_fields)
            file_name = company + '_income_statement.csv'
        elif statement_type == 'balance':
            fin_df = pd.DataFrame(balance_statement_list, columns=balance_fields)
            file_name = company + '_balance_statement.csv'
        
        fin_df = fin_df.set_index('financialYear')
        
        fin_df.to_csv(file_name)
        print(f'{file_name} was created')

TSLA_income_statement.csv was created
TSLA_balance_statement.csv was created
MSFT_income_statement.csv was created
MSFT_balance_statement.csv was created
AAPL_income_statement.csv was created
AAPL_balance_statement.csv was created
AMZN_income_statement.csv was created
AMZN_balance_statement.csv was created
META_income_statement.csv was created
META_balance_statement.csv was created


In [5]:
# Collaborate the company information into one datafile
companies = ['TSLA','MSFT','AAPL','AMZN','META']

# Create Empty Dataframe to add company information to
faang_financials_df = pd.DataFrame()

for company in companies:
    
    # Read in the Income and Balance Sheet Information
    income_df = pd.read_csv(company+'_income_statement.csv', index_col = 'financialYear')
    balance_df = pd.read_csv(company+'_balance_statement.csv', index_col = 'financialYear')
    
    # Combine the two financial statements by Year
    combined_financials_df = pd.merge(income_df, balance_df, on='financialYear')
    
    # Add Company Column and Index
    combined_financials_df['Company'] = company
    combined_financials_df = combined_financials_df.reset_index().set_index(['Company','financialYear'])
    
    # Add each company's financials to the new combined dataframe
    if faang_financials_df.empty:
        faang_financials_df = combined_financials_df
    else:
        faang_financials_df = pd.concat([faang_financials_df, combined_financials_df])

faang_financials_df.to_csv('faang_financials.csv')

In [6]:
combined_financials_df = pd.read_csv('faang_financials.csv', index_col=['Company','financialYear'])

# Convert all values $ to $Mil for readability
combined_financials_df = combined_financials_df/1000000

    # Change column names
columns = ['Revenue($m)','Cost_of_Goods_Sold($m)','R&D($m)','Operating_Costs($m)',
           'Net_Interest($m)','Net_Profit($m)','Cash_holdings($m)','Accounts_Receivable($m)',
           'Inventory($m)','Current Assets($m)','PPE($m)','Non-Current_Assets($m)',
           'Total_Assets($m)','Accounts_Payable($m)','Total_Liabilities($m)',
           'Retained_Earnings($m)','Equity($m)']

combined_financials_df.columns = columns

# Calculate Additional Fields
combined_financials_df['Gross_Profit($m)'] = combined_financials_df['Revenue($m)'] - combined_financials_df['Cost_of_Goods_Sold($m)']
combined_financials_df['Operating_Profit($m)'] = combined_financials_df['Gross_Profit($m)'] - combined_financials_df['Operating_Costs($m)']
    
# Calculate Financial Ratios
combined_financials_df['Gross_Profit_Ratio'] = round(combined_financials_df['Gross_Profit($m)'] / combined_financials_df['Revenue($m)'],4)

combined_financials_df['Net_Profit_Ratio'] = round(combined_financials_df['Net_Profit($m)'] / combined_financials_df['Revenue($m)'],4)

combined_financials_df['Operating_Profit_Ratio'] = round(combined_financials_df['Operating_Profit($m)'] / combined_financials_df['Revenue($m)'],4)

combined_financials_df['R&D:Revenue_ratio'] = round(combined_financials_df['R&D($m)'] / combined_financials_df['Revenue($m)'],4)

combined_financials_df['Current_Ratio%'] = (combined_financials_df['Cash_holdings($m)'] + combined_financials_df['Accounts_Receivable($m)'] +\
                                    combined_financials_df['Inventory($m)']) / combined_financials_df['Accounts_Payable($m)']

combined_financials_df['Quick_Ratio%'] = combined_financials_df['Cash_holdings($m)'] / combined_financials_df['Accounts_Payable($m)']

combined_financials_df['Debt_to_Equity_Ratio%'] = combined_financials_df['Total_Liabilities($m)'] / combined_financials_df['Equity($m)']

combined_financials_df['Return_On_Equity_Ratio%'] = combined_financials_df['Net_Profit($m)'] / combined_financials_df['Equity($m)']

combined_financials_df['Return_On_Assets_Ratio%'] = combined_financials_df['Operating_Profit($m)'] / combined_financials_df['Total_Assets($m)']

combined_financials_df.to_csv('faang_financials_clean.csv')

## ASX Companies

In [7]:
asx_companies = ['CBA.AX','WPC.AX','ANZ.AX','NAB.AX','MQG.AX']